# Project ClearChoise Textile Classifier


#### The aim of this project is to develop a pattern classifier. By inputting an image, the program will provide feedback on the pattern present in the photographed garment. Making mornings easier and more fun for those with visual impairments, helping them pick out their perfect outfit effortlessly!

- Model for flowers detection. flower_model_28-02-24.pt
- Model for solid colors. pattern_model_24-03-06.h5
- Model for jeans or not. jeans_model_24-03-06.h5
- Model for geometric patterns. geometric_24-03-04.h5

In [33]:
import cv2
import numpy as np
import torch
import tensorflow as tf
#import matplotlib.pyplot as plt
torch.__version__

'2.2.1+cpu'

If there is problem with yolo locally:

In [34]:
import pathlib
from pathlib import Path
pathlib.PosixPath = pathlib.WindowsPath

## Load Flowermodel

In [35]:
yolo_path = "C:/Users/emmel/Desktop/yolov5"
model_path = "./Flower_model/flower_model_28-02-24.pt"

yolo_model = torch.hub.load(yolo_path, 'custom', path=model_path, source='local')
yolo_model.conf = 0.30

YOLOv5  v6.1-362-g731a2f8c Python-3.10.6 torch-2.2.1+cpu CPU

Fusing layers... 


Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


## Load Patterns or not model

In [36]:
patterns_modelpath = "./solid_color_models/pattern_model_24-03-06.h5" 

patterns_model = tf.keras.models.load_model(patterns_modelpath)

## Load Jeansmodel

In [37]:
jeans_modelpath = "./solid_color_models/jeans_model_24-03-06.h5" 

jeans_model = tf.keras.models.load_model(jeans_modelpath)

labels_jeans = ["Not jeans", 
                "Jeans", 
]

## Load Geometricmodel

In [38]:
geometric_modelpath = "./geometric_model/geometric_24-03-04.h5" 

geometric_model = tf.keras.models.load_model(geometric_modelpath)

labels_geometric = ["check", 
                    "stripe", 
                    "dots"
]

### Test Images

In [39]:
img_paths = [ "./img_for_test/stripe_1.jpg",
                "./img_for_test/check_2.jpg",
                "./img_for_test/color_2.jpg",
                "./img_for_test/dots_2.jpg",
                "./img_for_test/flowers_2.jpg",
                "./img_for_test/jeans_2.jpg",
                "./img_for_test/test_41.jpg",
                ]

### Functions

In [40]:
def load_and_preprocess_image(image_path):
    img_dim = (64,64)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, img_dim)
    img = img[:, :, np.newaxis]
    img = img / 255
    return np.expand_dims(img, axis=0)

def predict_pattern_keras(img_preprocessed):
    pred_pattern = patterns_model.predict(img_preprocessed)
    pred_class_pattern = np.argmax(pred_pattern)

    if pred_class_pattern == 1:  # solid color
        predictions_jeans = jeans_model.predict(img_preprocessed)
        predicted_class_jeans = np.argmax(predictions_jeans)
        return labels_jeans[predicted_class_jeans]
        
    elif pred_class_pattern == 0:
        pred_geometric = geometric_model.predict(img_preprocessed)
        pred_class_geometric = np.argmax(pred_geometric)
        return labels_geometric[pred_class_geometric]
    else:
        return "Unknown Pattern"

# Function to predict pattern for images
def predict_pattern_for_images(img_paths):
    for img_path in img_paths:
        img = cv2.imread(img_path)
        yolo_result = yolo_model(img)
        yolo_preds = yolo_result.pandas().xyxy[0]
        
        if len(yolo_preds) > 0:
            print(f"{img_path}\nPredicted pattern: Flowers")
        else:
            img_preprocessed = load_and_preprocess_image(img_path)
            prediction = predict_pattern_keras(img_preprocessed)
            print(f"{img_path}\nPredicted pattern: {prediction}")

### Run

In [41]:
predict_pattern_for_images(img_paths)

1/1 [==============================] - 0s 109ms/step
./img_for_test/stripe_1.jpg
Predicted pattern: stripe
./img_for_test/check_2.jpg
Predicted pattern: Flowers
1/1 [==============================] - 0s 72ms/step
./img_for_test/color_2.jpg
Predicted pattern: Not jeans
1/1 [==============================] - 0s 29ms/step
./img_for_test/dots_2.jpg
Predicted pattern: dots
./img_for_test/flowers_2.jpg
Predicted pattern: Flowers
1/1 [==============================] - 0s 32ms/step
./img_for_test/jeans_2.jpg
Predicted pattern: Jeans
1/1 [==============================] - 0s 23ms/step
./img_for_test/test_41.jpg
Predicted pattern: Not jeans
